In [1]:
import pandas as pd
import numpy as np
import h5py
#from scipy.stats import pearsonr

In [2]:
# Load data
data_path = "/Users/guyshani/Documents/PHD/Aim_2/10x_data_mouse/20_1_2025__normalized/"
    
# Load expression matrix
#with h5py.File(data_path+'combined_normalized_data.h5', 'r') as f:
#    x_train = f['matrix'][:]

df=pd.read_csv(data_path+"combined_normalized_data.csv", sep=";")
meta=pd.read_csv(data_path+"combined_metadata.csv", sep=";")

/var/folders/g_/c_f20pc566n31j9p36lcdc0w0000gn/T/ipykernel_76983/1220664012.py:8: DtypeWarning: Columns (393,808,883,926) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(data_path+"combined_normalized_data.csv", sep=";")


In [14]:
celltype_count['B cells']

# Loop through keys and values
for key, value in celltype_count.items():
    print(f"{key}: {value}")

# Loop through just keys
for key in celltype_count.keys():
    print(key)
# Loop through just values
#for value in my_dict.values():
#    print(value)

B cells: 20966
Macrophages: 8690
T cells: 7866
Monocytes: 2198
Granulocytes: 866
NK cells: 713
Erythrocytes: 231
Dendritic cells: 38
Fibroblasts: 17
Endothelial cells: 3
B cells
Macrophages
T cells
Monocytes
Granulocytes
NK cells
Erythrocytes
Dendritic cells
Fibroblasts
Endothelial cells


In [11]:
## Load real and generated scRNA-seq data

run_dir = "/Users/guyshani/Documents/PHD/Aim_2/10x_data_mouse/20_1_2025__normalized/saved_models/run_20250121_150654_dataset_cell_type"
data_path = "/Users/guyshani/Documents/PHD/Aim_2/10x_data_mouse/20_1_2025__normalized/"

df_gen = pd.read_csv(f'{run_dir}_generated_data.csv')   # load generated data
cell_type = df_gen.iloc[:,-1]   # get cell type column
dataset = df_gen.iloc[:,-2] # get dataset column
df_gen = df_gen.drop(df_gen.columns[-2:], axis=1)   # drop label columns
df_real = pd.read_csv(f'{data_path}combined_normalized_data.csv', sep=";")  # load real data
cell_ids = df_real.iloc[:,0]    # get cell ids colums
gene_names = df_real.columns    # get gene names
#df_real = df_real.drop(df_real.columns[0], axis=1)

/var/folders/g_/c_f20pc566n31j9p36lcdc0w0000gn/T/ipykernel_78420/3186187728.py:10: DtypeWarning: Columns (393,808,883,926) have mixed types. Specify dtype option on import or set low_memory=False.
  df_real = pd.read_csv(f'{data_path}combined_normalized_data.csv', sep=";")  # load real data


In [ ]:
## replace "," with "." and convert string to float for the real data we loaded
df_real = df_real.replace(',', '.', regex=True)
df_real = df_real.apply(pd.to_numeric)
df_real = df_real.to_numpy()
df_gen = df_gen.to_numpy()
# test that both matrices have the same size
print("x_test shape:", df_real.shape)
print("x_gen shape:", df_gen.shape)

x_test shape: (41588, 1000)
x_gen shape: (41588, 1000)


In [7]:
def upper_diag_list(m_):
   """Returns upper triangular values of matrix, excluding diagonal"""
   # np.triu_indices gets indices of upper triangle, k=1 excludes diagonal
   return m_[np.triu_indices(len(m_), k=1)]

def standardize(a):
    """Standardizes data by subtracting mean and dividing by std dev (z-score)"""
    a_off = np.mean(a, axis=0)  # Calculate mean
    a_std = np.std(a, axis=0)   # Calculate standard deviation
    S = (a - a_off) / a_std     # Z-score standardization
    S[np.isnan(S)] = (a - a_off)[np.isnan(S)]  # Handle zeros in std
    return S
def pearson_correlation(x: np.array, y: np.array):
    """Computes correlation between each pair of vectors in x and y matrices"""
    assert x.shape[0] == y.shape[0] # Check input matrices have same number of samples
    x_ = standardize(x)
    y_ = standardize(y)
    return np.dot(x_.T, y_) / x.shape[0]  # Compute correlation as dot product of standardized values

def correlations_list(x: np.array, y: np.array):
    """Gets list of correlations between all pairs"""
    corr = pearson_correlation(x, y)
    return upper_diag_list(corr)

def gamma_coeff_score(x_test: np.array, x_gen: np.array):
    """Calculates correlation between distance matrices of test and generated data"""
    assert x_test.shape[1] == x_gen.shape[1]    # Verify matrices have same number of features
    dists_x = 1 - correlations_list(x_test, x_test)
    dists_y = 1 - correlations_list(x_gen, x_gen)
    # Reshape for pearson_correlation, to column vectors and compute correlation between distances
    return pearson_correlation(dists_x.reshape(-1, 1), dists_y.reshape(-1, 1))

In [8]:
g = gamma_coeff_score(df_real, df_gen)